#  IBM Applied D.S Capstone Project (Battle of Neighborhoods)


## Determining the best place to open a new business in outter side of London city limits.


London is definitely one of the best places to start a business. Even though it has high competition it still offers plenty of opportunities if you have a lucrative idea. Being the capital of both England and the UK, it is the center for commerce, finance, fashion, healthcare, research, development, business, tourism, and plenty more. As such there are many opportunities for people in London that want to start a business. But It is also an Expensive place to Live and thrive. Many businesses want to open a venue here and many pay top price to have their window on Piccadilly or Oxford Street. But what about those, 
        
    1. Who want to start their own business and cannot really afford to open in the City yet? 
    2. Where is it best to open a new place? 
    3. Where will it be cheapest and will have enough people living around to be popular? 
    4. Where the competition is not too overwhelming?

Lets find out the answers for the above questions:





## Content in Notebook

<font color=blue> 1. Import Libraries </font>

<font color=blue> 2. Scrap data from webpage into a DataFrame </font>

    
<font color=blue> 3. Exploratory Visualization 1 </font>

    


<font color=Red> Target Audience</font>

1. First time enterpreneurs, who want to start their first business. Below dataset will give a comprehensive insight into where best to open a new venue, to maximise the value for money.

2. People who already run a business and want to start a new branch. Given the extra information, it may provide some valuable information before decision     making.

# 1. Import Libraries

In [7]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd  # library to handle data in a vectorized manner
import numpy as np
import requests

print("Libraries imported.")

Libraries imported.


## 2. Scrap data from webpage into a DataFrame
Name of the London Boroughs, area and population from web scrapping. Filter the boroughs to get the best candidates for the analysis.

In [12]:
## Get the names of Boroughs, Areas, Population, Coordinates from Wikipedia.   
url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

df=pd.read_html(url, header=0)[0]

df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


Processing the Information From Wiki To Make Necessary Lists. Check and Compare with Google Search and Refine if Necessary

In [13]:
# we get the nessessary information on London Boroughs, dropping the extras, that will not be needed for the analysis.

clean_df = df.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
clean_df['Inner'].replace(np.nan,'0', inplace=True)
clean_df['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
clean_df['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
clean_df['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
clean_df.head()
clean_df['Inner'] = '0'
clean_df.head()

,Borough,Inner,Area (sq mi),Population (2019 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Barnet,0,33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Then we rename the columns, making the dataset better on the eyes. Because of extra notes in the Wiki page, we will rename some of the Boroughs. Due to the staggering difference in rent price, as well as the amount of venues in London, we will filter to have only the Outer boroughs going forward

In [14]:
clean_df['Inner'] = clean_df.Borough.isin(Inn).astype(int)
Out = clean_df[clean_df.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
ldf = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2013 est)[1]":"Population"})
geolocator = Nominatim(user_agent="London_explorer")
ldf['Co-ordinates']= ldf['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
ldf[['Latitude', 'Longitude']] = ldf['Co-ordinates'].apply(pd.Series)
ldf.head()

,Borough,Area,Population (2019 est)[1],Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814


Finally, we edit the coordinates, that are incorrect.
Lets filter the set even more - because rent price is a big issue for the new business, we will get the boroughs with the lowest ones.

In [16]:
Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
ldf['Max_Rent'] = Max_Rent

ldf["Max_Rent"] = pd.to_numeric(ldf["Max_Rent"])
fin = ldf[ldf.Max_Rent <= 125]
fin

Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]

fin = fin.drop(['Co-ordinates', 'Longitude'], axis=1)

fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, 39.9692378, 51.4028046, 51.6520851, 51.587929849999995, 51.5443851, 51.41086985, 51.5763203]
Old Longitude list:  [0.15050434261994267, -82.936864, 0.0148142, -0.0810175, -0.10541010599099046, -0.14430716398919305, -0.18809708858824303, 0.0454097]


,Borough,Area,Population (2019 est)[1],Latitude,Max_Rent,Longitude
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504
2,Bexley,23.38,248287,39.969238,97.00,-82.936864
4,Bromley,57.97,332336,51.402805,118.50,0.014814
8,Enfield,31.74,333794,51.652085,102.25,-0.081018
12,Haringey,11.42,268647,51.587930,107.75,-0.105410
14,Havering,43.35,259552,51.544385,86.00,-0.144307
22,Merton,14.52,206548,51.410870,123.75,-0.188097
24,Redbridge,21.78,305222,51.576320,118.50,0.045410


Now lets make a map, with all boroughs in question marked up

## 3. Exploratory Visualization 1

In [17]:
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [18]:
!pip install folium
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], 
                            fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
